# Handling Authentication

- APIs often require authentication to control access, rate limits, and auditing.
- Without authentication, requests to protected endpoints will fail with codes like 401 (Unauthorized) or 403 (Forbidden).
- This section demonstrates a simple GET to a protected endpoint, illustrating why auth is needed.

## Why Authentication?

- Authentication tells the API who you are, enabling personalized data and higher rate limits.
- It prevents unauthorized access to private resources and supports auditing of actions.
- Authenticated requests often succeed where anonymous requests would be blocked or limited.

In [1]:
GITHUB_ENDPOINT = "https://api.github.com"
HTTPBIN_ENDPOINT = "https://httpbin.org"

In [2]:
import requests 

urls = {
    "public_endpoint": f"{GITHUB_ENDPOINT}/zen",
    "protected_endpoint": f"{GITHUB_ENDPOINT}/user",

}

for description, url in urls.items():
    res = requests.get(url, timeout=5)
    print(f"{description} ({url}) : {res.status_code}")
    print(res.text[:200])

public_endpoint (https://api.github.com/zen) : 200
Keep it logically awesome.
protected_endpoint (https://api.github.com/user) : 401
{
  "message": "Requires authentication",
  "documentation_url": "https://docs.github.com/rest",
  "status": "401"
}


## Basic Authentication

- Basic Auth sends a username and password with each request, encoded in the `Authorization` header.
- `requests` accepts an `auth=(username, password)` tuple and handles encoding automatically.
- Servers return `401 Unauthorized` when credentials are missing or incorrect.

In [7]:
import requests
import json

url = f"{HTTPBIN_ENDPOINT}/basic-auth/myuser/mypassw"

try:
    res = requests.get(url, auth=("myuser", "mypasswd") ,timeout=10)
    res.raise_for_status()  # 4xx or 5xx errors will be raised
    print(f"Status code: {res.status_code}")
    print("Response JSON:")
    print(json.dumps(res.json()))
except requests.exceptions.HTTPError as e:  
    print(f"HTTP error occurred: {e}")

HTTP error occurred: 401 Client Error: UNAUTHORIZED for url: https://httpbin.org/basic-auth/myuser/mypassw


## Token-Based Authentication

- Modern APIs use API keys or bearer tokens passed via the `Authorization` header.
- For GitHub PATs, use `Authorization: token <PAT>` or `Authorization: Bearer <PAT>`; for OAuth2, `Authorization: Bearer <token>`.
- Always load tokens from environment variables to avoid hardcoding secrets.

In [11]:
import os
import requests
from dotenv import load_dotenv

load_dotenv(override=True)

token = os.getenv("GH_PAT", "")
print(f"Token: {token[:15]}...")

urls = {
    "public_endpoint": f"{GITHUB_ENDPOINT}/zen",
    "protected_endpoint": f"{GITHUB_ENDPOINT}/user",

}

for description, url in urls.items():
    try:
        headers = {
            "Authorization": f"token {token}"
        }
        res = requests.get(url, headers=headers,timeout=5)
        res.raise_for_status()  # 4xx or 5xx errors will be raised
        print(f"Status code: {res.status_code}")
        print(f"Authenticated user: {res.json().get('login')}")
    except requests.JSONDecodeError as e:
        print(f"Invalid JSON in response body. Defauting to text:")
        print(res.text[:200])        
    except requests.exceptions.HTTPError as e:  
        print(f"HTTP error occurred: {e}")

Token: github_pat_11AC...
Status code: 200
Invalid JSON in response body. Defauting to text:
Approachable is better than simple.
Status code: 200
Authenticated user: shubheshswain91


## Common Pitfalls & How to Avoid Them

- Using the wrong header format (e.g., `Bearer` vs `token`) causes 401/403 errors. Follow API docs.
- Hardcoding secrets risks accidental exposure; always use environment variables or secret managers.